In [1]:
#importing packages
from collections import Counter, defaultdict
import math
import numpy as np
import os
import pandas as pd
import re
from scipy.sparse import csr_matrix
#import urllib.request
#import zipfile

# Use nltk for valid words
import nltk
# Need to make hash 'dictionaries' from nltk for fast processing
import collections as co


In [2]:
# Read the input
d = pd.read_csv("F:\IITC\ADM\Extra\consumer_complaints.csv") # the consumer dataset is now a Pandas DataFrame
# Only interested in data with consumer complaints
d = d[d['Consumer complaint narrative'].notnull()]

C:\Users\Swapnil\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(d['Consumer complaint narrative'])

56777     Received Capital One charge card offer XXXX. A...
56834     I do n't know how they got my cell number. I t...
56884     I 'm a longtime member of Charter One Bank/RBS...
56894     After looking at my credit report, I saw a col...
56898     I received a call from a XXXX XXXX from XXXX @...
56910     Was not contacted 4 years later about some pri...
56911     Collection Consultants is reporting a collecti...
56914     I had my purse stolen in 2007. They never foun...
56917     I attempted to apply for a Discover Card Onlin...
56918     Continued attempts by XXXX XXXX XXXX to collec...
56920     This is a continuation of a previous issue wit...
56928     Going through a divorce, my ex and I were unab...
56930     I am assisted with my mortgage through an agen...
56939     i submitted a payment on XXXX XXXX 2015 for my...
56940     I recieved a notice from Midland Credit Manage...
56956     XXXX Card services was bought out by Capital O...
56965     I was reported late by Discove

In [4]:
print(d[1:5])

      Date received           Product                            Sub-product  \
56834     3/23/2015   Debt collection  Other (i.e. phone, health club, etc.)   
56884     3/23/2015       Credit card                                    NaN   
56894     3/23/2015  Credit reporting                                    NaN   
56898     3/23/2015   Debt collection                            Payday loan   

                                        Issue  \
56834     Improper contact or sharing of info   
56884                                 Rewards   
56894  Incorrect information on credit report   
56898     Improper contact or sharing of info   

                                   Sub-issue  \
56834      Contacted me after I asked not to   
56884                                    NaN   
56894                Information is not mine   
56898  Talked to a third party about my debt   

                            Consumer complaint narrative  \
56834  I do n't know how they got my cell number. I 

In [42]:
def tokenize_string(my_string):
    """ DONE. You should use this in your tokenize function.
    """
    #return re.findall('[\w\-]+', my_string.lower())
    #\W -> Matches any non-alphanumeric character; 
    #this is equivalent to the class [^a-zA-Z0-9_]. 
      
    # preprocessing ->
    # remove numbers    
    word_list = re.sub('[^a-zA-Z]+', ' ', my_string.lower()).split()
    
   
    # remove stop words
    stopWords = co.Counter( nltk.corpus.stopwords.words() )

    #print(stopWords)
    
    clean_word_list = [x for x in word_list if x not in stopWords]
    
    #print(clean_word_list)
    
    
    # remove small and large words
   
    word_list = [x for x in clean_word_list if len(x) > 4 and len(x) < 15 ]
    
    
    # remove xxxx
    
    clean_word_list = [x for x in word_list if "xxxx" not in x ]
   
    # remove too low and too high frequency words -> we can't calculate here

    
    
    return(clean_word_list)

In [18]:
def tokenize(data):
    """
    Append a new column to the movies DataFrame with header 'tokens'.
    This will contain a list of strings, one per token, extracted
    from the 'genre' field of each movie. Use the tokenize_string method above.
    Note: you may modify the movies parameter directly; no need to make
    a new copy.
    Params:
      movies...The movies DataFrame
    Returns:
      The movies DataFrame, augmented to include a new column called 'tokens'.
    >>> movies = pd.DataFrame([[123, 'Horror|Romance'], [456, 'Sci-Fi']], columns=['movieId', 'genres'])
    >>> movies = tokenize(movies)
    >>> movies['tokens'].tolist()
    [['horror', 'romance'], ['sci-fi']]
    """
    ###TODO
    
    # step 1 -> do tokenize_string for each row in movies['genres']

    all_words = []
    for row in data['Consumer complaint narrative']:
        #genre_list = re.sub(r'[||)|(]', r' ',row.lower()).split()
        word_list = tokenize_string(row)
        #print(word_list)
        #print(len(genre_list))
        all_words.append(word_list)
    
    # step 2 -> add column tokens in movies
    array = np.array(all_words)
    
    #print(array[:5])
    #print('#list = ',len(array))
    
    new_data = d.assign(tokens = array)
    
    #print(new_movies.head(5))
    return(new_data)  
    pass

In [38]:
def featurize(data):
    """
    Append a new column to the movies DataFrame with header 'features'.
    Each row will contain a csr_matrix of shape (1, num_features). Each
    entry in this matrix will contain the tf-idf value of the term, as
    defined in class:
    tfidf(i, d) := tf(i, d) / max_k tf(k, d) * log10(N/df(i))
    where:
    i is a term
    d is a document (movie)
    tf(i, d) is the frequency of term i in document d
    max_k tf(k, d) is the maximum frequency of any term in document d
    N is the number of documents (movies)
    df(i) is the number of unique documents containing term i
    Params:
      movies...The movies DataFrame
    Returns:
      A tuple containing:
      - The movies DataFrame, which has been modified to include a column named 'features'.
      - The vocab, a dict from term to int. Make sure the vocab is sorted alphabetically as in a2 (e.g., {'aardvark': 0, 'boy': 1, ...})
   
    """
    ###TODO
    #print(movies[:5]) 
    
    #step 1 -> build a vocab and df(term)
    vocab = {}
    vocab_list = []
    df = {}
    
    for row1 in data['tokens']:
        row = list(set(row1))
        for term in row:
            if term not in vocab.keys():
                vocab.setdefault(term,-1)
             
            if term not in df.keys(): 
                df.setdefault(term,1)
            else :
                df[term] += 1
             
             
    #print('vocab = ', vocab)
    
    vocab_list = sorted(vocab.keys(), key = lambda x:x)
    #print('vocab_list = ', vocab_list)
    
    for i,term in enumerate(vocab_list):
         vocab[term] = i
            
    #        
         
    #print('Sorted vocab = ', sorted(vocab.items()))
    #print('df = ',sorted(df.items(), key=lambda x:x[0]))
    
    # step 2 -> Build a csr_matrix for each row of movies['tokens']
    N = len(data)
    #print('N = ',N)
    
    #[comedy, comedy, comedy, horror]  -> max_k tf(k, d) = 3 
    #[action, comedy,thriller] -> tf(action, d) =1
    # df(i) ->
    #num_features is the total number of unique features across all documents.
    
    csr_array =[]
    
    for row1 in data['tokens']:
        csr_row = []
        csr_col = []
        csr_data = []
        max_k = 0
       
        #print(row1)
        
        if row1 != []:
            max_k = Counter(row1).most_common()[:1][0][1]
            row = list(set(row1))

            #print(max_k)

            #print('removed duplicates =',row)
            for term in row:       
                csr_row.append(0)
                csr_col.append(vocab[term])
                #tfidf(i, d) := tf(i, d) / max_k tf(k, d) * log10(N/df(i))
                tf = Counter(row1)[term]
                #max_k = max_k.most_common()[:1][0][1]
         
                #print('term = %s ---> tf = %d ---> max_k = %d'%(term,tf,max_k))
                tfidf = (tf / max_k) * math.log10(N/df[term])
                csr_data.append(tfidf)
           

         
        X = csr_matrix((csr_data, (csr_row, csr_col)), shape=(1, len(vocab)), dtype=np.float64)
       
        #print('X ->\n',X.toarray())
        csr_array.append(X)
    

    # step 3 -> add column features to movies 
    #print('size of csr_array = ',len(csr_array)) 
    #print('CSR = ',csr_array[:2])  
    new_data = data.assign(features = csr_array)
    #print(new_movies.head(2))
     
    return(new_data,vocab)   
    
    pass

In [8]:
data = tokenize(d)

In [12]:
print(data['tokens'][1:5])

56834    [know, got, cell, number, told, would, deal, o...
56884    [longtime, member, charter, one, bank, rbs, ci...
56894    [looking, credit, report, saw, collection, acc...
56898    [received, call, xxxx, xxxx, xxxx, xxxx, ext, ...
Name: tokens, dtype: object


In [39]:
data, vocab = featurize(data)

In [40]:
print('vocab:')
print(sorted(vocab.items())[:500])

vocab:
[('aa', 0), ('aaa', 1), ('aaaa', 2), ('aaaaan', 3), ('aaaaargh', 4), ('aaadvantage', 5), ('aaaked', 6), ('aaarm', 7), ('aab', 8), ('aac', 9), ('aacceptance', 10), ('aaccount', 11), ('aaccounts', 12), ('aacount', 13), ('aadavantage', 14), ('aaddress', 15), ('aadhar', 16), ('aadvantage', 17), ('aadvatage', 18), ('aafes', 19), ('aafraid', 20), ('aag', 21), ('aagain', 22), ('aagainst', 23), ('aagon', 24), ('aaii', 25), ('aaliant', 26), ('aallc', 27), ('aallcdelete', 28), ('aalthough', 29), ('aamco', 30), ('aamerican', 31), ('aams', 32), ('aand', 33), ('aapproved', 34), ('aapproximatly', 35), ('aaproximately', 36), ('aarea', 37), ('aargon', 38), ('aarmr', 39), ('aaron', 40), ('aarons', 41), ('aaround', 42), ('aarp', 43), ('aarrearge', 44), ('aas', 45), ('aasigned', 46), ('aasked', 47), ('aasking', 48), ('aastra', 49), ('aat', 50), ('aatached', 51), ('aattempting', 52), ('aaudulent', 53), ('aavantage', 54), ('aavings', 55), ('ab', 56), ('aba', 57), ('aback', 58), ('abad', 59), ('abail

In [41]:
print(len(vocab))

63974
